<a href="https://colab.research.google.com/github/alongutm/Harry-Potter-NLP/blob/main/Harry_Potter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_lg  # Important! you need to restart runtime after install

In [1]:
!pip install turicreate
!pip install kaggle 
!pip install gensim
!pip install pyLDAvis
!pip install spaCy
!pip install afinn
!pip install nltk
!pip install plotly_express

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!pip install -Uqq ipdb
import ipdb

In [45]:
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline


nlp = spacy.load('en_core_web_lg')
book_file_path = "/content/Harry Potter and the Sorcerer's Stone.txt"
book2_file_path = "/content/Harry Potter and the Deathly Hallows.txt"
with open(book2_file_path, 'r') as book_file:
  text = book_file.read()
text = text.replace("\n", " ")
doc = nlp(text[:300000])
stopwords = set(STOPWORDS) | {'Yeh'}

In [46]:
def get_entities(document):
  entities_dict= {}
  for entity in document.ents:
    label = entity.label_
    entity_lower = entity.text.lower().title()
    if  label not in entities_dict:
        entities_dict[label] = {entity_lower: 0}
    if entity_lower not in entities_dict[label]:
      entities_dict[label][entity_lower] = 1
    else:
      entities_dict[label][entity_lower] += 1

  stopwords = set(STOPWORDS) | {'Yeh'}
  entities_counts = {k: v for k, v in entities_dict["PERSON"].items() if v > 5 and k not in stopwords}
  entites_counts_sorted = sorted(entities_counts.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in entites_counts_sorted]

print(get_entities(doc))  

['Harry', 'Hermione', 'Ron', 'Dumbledore', 'Voldemort', 'Snape', 'Weasley', 'Hagrid', 'Lupin', 'Luna', 'Kreacher', 'Bill', 'Griphook', 'Ginny', 'Fleur', 'Fred', 'Xenophilius', 'Potter', 'George', 'Tonks', 'Greyback', 'Scrimgeour', 'Mcgonagall', 'Bellatrix', 'Harry Potter', 'Lily', 'Neville', 'Muriel', 'Dudley', 'Yaxley', 'Gregorovitch', 'Kingsley', 'Malfoy', 'Regulus', 'Doge', 'Dobby', 'Travers', 'Draco', 'Horcruxes', 'Dean', 'Krum', 'Ariana', 'Uncle Vernon', 'Umbridge', 'Aberforth', 'Mundungus', 'Godric', 'Percy', 'Lovegood', 'Cattermole', 'Crabbe', 'Ted', 'Lucius', 'Albus', 'James', 'Phineas Nigellus', 'Ollivander', 'Severus', 'Skeeter', 'Goyle', 'Horcrux', 'Wormtail', 'Sirius', 'Kendra', 'Hedwig', 'Rita Skeeter', 'Scabior', 'Lucius Malfoy', 'Narcissa', 'Albus Dumbledore', 'Delacour', 'Dirk', 'Dursleys', 'Remus', 'Dedalus', 'Arthur', 'Burrow', 'Flitwick', 'Snatchers', 'Mum', 'Charlie', 'Amycus', 'Disapparated', 'Ravenclaws', 'Dad', 'Moody', 'Lee', 'Severus Snape', 'Prophet', 'Vernon 

In [47]:
# TODO add function to merge duplicate names such as 'Potter' and 'Harry Potter'

In [48]:
from nltk.sentiment import SentimentIntensityAnalyzer

def polarity_score(sentence):
  sia = SentimentIntensityAnalyzer()
  score = sia.polarity_scores(sentence)
  return score['pos'], score['neg'] ,score['neu']

In [54]:
import re

def normalize_text(text):
    """
    Returns a normalized version of the specified string.
    You can add default parameters as you like (they should have default values!)
    You should explain your decisions in the header of the function.
    Args:
        text (str): the text to normalize
    Returns:
        string. the normalized text.
    """
    words_list = re.findall(r'(?x)(?:[A-Z]\.)+|\w+(?:-\w+)*|\$?\d+(?:\.\d+)?%?', text.lower())  # a regex for extracting tokens from the text
    normalized_text = ""
    for word in words_list:
        normalized_text += f"{word} "

    return normalized_text


def get_ngrams(txt, characters_list, n=10, threshold=5):
    characters_dict = {}
    characters_list_lower_case = [x.lower() for x in characters_list]
    dic_index_to_sentiment = {0: 'Positive', 1: 'Negative', 2: 'Neutral'}
    related_characters = []
    ngrams_to_evaluate_sentiment = []
    txt = normalize_text(txt).split()
    ngrams = nltk.FreqDist(nltk.ngrams(txt, n))
    for i in range(0, len(characters_list_lower_case), 1):
        for j in range(i+1, len(characters_list_lower_case), 1):
            counter = 0
            sum_sentiments = [0, 0]
            for ngram in ngrams:
                if characters_list_lower_case[i] in ngram and characters_list_lower_case[j] in ngram:
                    counter += 1
                    temp_score = polarity_score(' '.join(ngram))
                    # print("temp score :", temp_score)
                    sum_sentiments[0] += temp_score[0]
                    sum_sentiments[1] += temp_score[1]

            max_value = max(sum_sentiments)
            index_of_max_value = sum_sentiments.index(max_value)

            characters_dict[(characters_list_lower_case[i], characters_list_lower_case[j])] = (dic_index_to_sentiment[index_of_max_value], counter)

    # print("keys: " ,characters_dict.keys())
    for pair in characters_dict.keys():
      print(characters_dict[pair][1])      
      if characters_dict[pair][1] >= threshold:
        related_characters.append((pair, characters_dict[pair][0]))

    return related_characters


def get_ngrams_judes_style(txt, characters_list, n=10, threshold=5):
    characters_dict = {}
    characters_list_lower_case = [x.lower() for x in characters_list]
    dic_index_to_sentiment = {0: 'Positive', 1: 'Negative', 2: 'Neutral'}
    related_characters = []
    ngrams_to_evaluate_sentiment = []
    txt = normalize_text(txt).split()
    ngrams = nltk.FreqDist(nltk.ngrams(txt, n))
    for i in range(0, len(characters_list_lower_case), 1):
        for j in range(i+1, len(characters_list_lower_case), 1):
            counter = 0
            sum_sentiments = [0, 0]
            for ngram in ngrams:
                if characters_list_lower_case[i] in ngram and characters_list_lower_case[j] in ngram:
                    counter += 1
                    temp_score = polarity_score(' '.join(ngram))
                    if temp_score[0] > temp_score[1]:
                      sum_sentiments[0] += 1
                    elif temp_score[0] < temp_score[1]:
                      sum_sentiments[1] += 1

            max_value = max(sum_sentiments)
            index_of_max_value = sum_sentiments.index(max_value)
            characters_dict[(characters_list_lower_case[i], characters_list_lower_case[j])] = (dic_index_to_sentiment[index_of_max_value], counter)

    # print("keys: " ,characters_dict.keys())
    for pair in characters_dict.keys():
      print(characters_dict[pair][1])      
      if characters_dict[pair][1] >= threshold:
        related_characters.append((pair, characters_dict[pair][0]))

    return related_characters

In [55]:
list_ent = get_entities(doc)
get_ngrams(text, list_ent)

Streaming output truncated to the last 5000 lines.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
7
0
0
12
0
0
0
8
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
8
0
0
0
0
6
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
48
0
0
0
0
0
0
6
0
0
0
0
4
0
0
0
0
0
0
0
0
0
7
0
18
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
17
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
52
0
0
0
0
0
15
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
9
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
42
0
0
0
0
0
0
0
0
0
0
12
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
18
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12
0
0

[(('harry', 'hermione'), 'Negative'),
 (('harry', 'ron'), 'Negative'),
 (('harry', 'dumbledore'), 'Positive'),
 (('harry', 'voldemort'), 'Negative'),
 (('harry', 'snape'), 'Negative'),
 (('harry', 'weasley'), 'Negative'),
 (('harry', 'hagrid'), 'Negative'),
 (('harry', 'lupin'), 'Negative'),
 (('harry', 'luna'), 'Positive'),
 (('harry', 'kreacher'), 'Negative'),
 (('harry', 'bill'), 'Negative'),
 (('harry', 'griphook'), 'Positive'),
 (('harry', 'ginny'), 'Negative'),
 (('harry', 'fleur'), 'Positive'),
 (('harry', 'fred'), 'Positive'),
 (('harry', 'xenophilius'), 'Negative'),
 (('harry', 'potter'), 'Positive'),
 (('harry', 'george'), 'Negative'),
 (('harry', 'tonks'), 'Positive'),
 (('harry', 'greyback'), 'Positive'),
 (('harry', 'scrimgeour'), 'Negative'),
 (('harry', 'mcgonagall'), 'Negative'),
 (('harry', 'bellatrix'), 'Negative'),
 (('harry', 'lily'), 'Positive'),
 (('harry', 'neville'), 'Positive'),
 (('harry', 'muriel'), 'Negative'),
 (('harry', 'dudley'), 'Positive'),
 (('harry',

In [51]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='black', 
                stopwords = stopwords, 
                min_font_size = 10,
                collocations=False)
wordcloud.generate_from_frequencies(frequencies=dict(entites_counts_sorted))
plt.figure(figsize = (20, 20), facecolor = None) 
plt.imshow(wordcloud)

NameError: ignored

In [ ]:
nltk.download(["vader_lexicon"])